# Local Navigation 

### Thymio connection

In [2]:
import tdmclient.notebook
await tdmclient.notebook.start()

### Import libraries

In [3]:
import math
import numpy as np
import serial
import time

In [4]:
await tdmclient.notebook.stop()

In [5]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 7ceccd72-4650-4181-812c-177085571dc0

In [6]:
await node.wait_for_variables()
node.var

{'_fwversion': [14, 0],
 '_id': [-1654],
 '_imot': [0, 0],
 '_integrator': [0, 0],
 '_productId': [8],
 '_vbat': [906, 911],
 'acc': [0, 0, 21],
 'acc._tap': [0],
 'button.backward': [0],
 'button.center': [0],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [11433, 9820, 14837, 14744, 16120],
 'buttons._noise': [118, 106, 124, 110, 132],
 'buttons._raw': [11465, 9814, 14845, 14747, 16146],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-1654],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [256],
 'mic.intensity': [4],
 'mic.threshold': [0],
 'motor.left.pwm': [0],
 'motor.left.speed': [-4],
 'motor.left.target': [0],
 'motor.right.pwm': [0],
 'motor.right.speed': [0],
 'motor.right.target': [0],
 'prox.comm.

In [7]:
# Connect to the Thymio robot
client.tdm_addr
client.tdm_port
th = node

### Desciption
Local navigation allows modulating the trajectory to avoid unforeseen, local obstacles. It often pushes the controller to leave the optimal path to make an avoidance manoeuvre. Once the obstacle is passed, the controller can go back to the optimal path or find a new one. An important issue is to decide when the controller has to start avoiding, and, more difficult, when the obstacle can be considered as passed.

### Parameters 

In [8]:
## Parameters for local navigation
pi = math.pi

## Global variables
speed0 = 100       # nominal speed
speedGain = 2      # speed gain with obstacle
obstThrL = 10      # low obstacle threshold to switch state 1->0
obstThrH = 20     # high obstacle threshold to switch state 0->1
# obstSpeedGainLocal = 5  # /100 (actual gain: 5/100=0.05)
obstSpeedGainGlobal = [6, 4, -2, -6, -8]  # /100

state = 1          # 0=towards goal, 1=obstacle avoidance
obst = [0,0]       # measurements from left and right prox sensors

timer_period[0] = 10   # 10ms sampling time

### Sensor readings

In [9]:
await node.wait_for_variables()
for i in range(10):
    # You can also replace "prox.horizontal" by an other prox sensor:
    print(list(node["prox.horizontal"]))
    await client.sleep(0.2)

[2065, 1419, 0, 2104, 2946, 0, 0]
[2048, 1356, 0, 2055, 2609, 0, 0]
[2075, 1459, 0, 2269, 2493, 0, 0]
[2114, 1730, 2574, 3305, 3430, 0, 0]
[2094, 1473, 3252, 4256, 4335, 0, 0]
[2077, 1458, 3670, 4570, 4605, 0, 0]
[2070, 1477, 3869, 4714, 4687, 0, 0]
[2069, 1622, 1831, 2505, 2722, 0, 0]
[2068, 1502, 0, 0, 2027, 0, 0]
[1971, 0, 0, 0, 0, 0, 0]


In [10]:
# Reading sensor measurements
# sensor_prox = th["prox.horizontal"]
await node.wait_for_variables({"prox.horizontal"})
for i in range(10):
    sensor_prox = node.v.prox.horizontal 
    print(list(sensor_prox))
    await client.sleep(0.2)

[1913, 0, 0, 0, 0, 0, 0]
[1944, 0, 0, 0, 0, 0, 0]
[1976, 0, 0, 0, 0, 0, 0]
[2023, 1544, 0, 0, 0, 0, 0]
[2108, 1725, 1652, 0, 0, 0, 0]
[1692, 0, 0, 0, 1672, 0, 0]
[0, 0, 0, 0, 1440, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1486, 0, 0]
[0, 0, 0, 0, 0, 0, 0]


### Functions

In [11]:
aw(node.lock())

Node 7ceccd72-4650-4181-812c-177085571dc0

In [12]:
@onevent
def move_forward(speed0):
#     global motor_left_target, motor_right_target
    v = {
        "motor.left.target": [speed0],
        "motor.right.target": [speed0],
    }
    aw(node.set_variables(v))

@onevent
def stop_moving():
#     global motor_left_target, motor_right_target
    x = {
        "motor.left.target": [0],
        "motor.right.target": [0],
    }
    aw(node.set_variables(x))

### Method 1
#### 1. ANN
- Train to find appropriate weights for each sensor

#### 2. Potential Field Navigation

In [18]:
@onevent 
def pt_field(sensor_prox):
    global speed0, speedGain, obstSpeedGainGlobal
    
    # speed based on nominal velocity and ground
    spLeft = speed0 - speedGain 
    spRight = speed0 + speedGain
    
    # adjustment for obstacles
    for i in range(5):
        print(sensor_prox[0])
        spLeft += sensor_prox[i] * obstSpeedGainGlobal[i] // 100
        spRight += sensor_prox[i] * obstSpeedGainGlobal[4 - i] // 100
    
    # motor control
    y = {
        "motor.left.target": [spLeft],
        "motor.right.target": [spRight],
    }
    aw(node.set_variables(y))
#     motor_left_target = spLeft
#     motor_right_target = spRight
    
    # Move in the same direction for 2 more seconds
#     move_forward(speed0)
#     time.sleep(0.2)
#     stop_moving()
    
#      # Continue towards goal
#     move_forward(speed0)   

In [23]:
# Test
# acquisition from the proximity sensors to detect obstacles
await node.wait_for_variables({"prox.horizontal"})

for i in range(100): 
    sensor_prox = node["prox.horizontal"]
#     sensor_prox = node.v.prox.horizontal 
    print(list(sensor_prox))
    obst = [sensor_prox[0], sensor_prox[4]]

    if state == 0: 
        # switch from goal tracking to obst avoidance if obstacle detected
        if (obst[0] > obstThrH):
            state = 1
        elif (obst[1] > obstThrH):
            state = 1
    elif state == 1:
        if obst[0] < obstThrL:
            if obst[1] < obstThrL : 
                # switch from obst avoidance to goal tracking if obstacle got unseen
                # move 2 more seconds in the same direction
                state = 0
                move_forward(speed0)
                time.sleep(2)
                stop_moving()
    if  state == 0 :
        # goal tracking: stop for now
        leds_top = [0,0,0]
        stop_moving()
    else:
        leds_top = [30,30,30]
        pt_field(sensor_prox)
        # obstacle avoidance: accelerate wheel near obstacle
        # motor_left_target = speed0 + obstSpeedGain * (obst[0] // 100)
        #motor_right_target = speed0 + obstSpeedGain * (obst[1] // 100)

    await client.sleep(0.2) # Buffer for Thymio to read sensor readings

[0, 0, 0, 0, 0, 0, 0]
[0, 2415, 4315, 0, 0, 0, 0]
[0, 2419, 4293, 0, 0, 0, 0]
[0, 2428, 4299, 0, 0, 0, 0]
[0, 2412, 4296, 0, 0, 0, 0]
[0, 2415, 4298, 0, 0, 0, 0]
[0, 2418, 4296, 0, 0, 0, 0]
[0, 2428, 4300, 0, 0, 0, 0]
[0, 2424, 4300, 0, 0, 0, 0]
[0, 2419, 4296, 0, 0, 0, 0]
[0, 2434, 4293, 0, 0, 0, 0]
[0, 2421, 4298, 0, 0, 0, 0]
[0, 2427, 4294, 0, 0, 0, 0]
[0, 2420, 4300, 0, 0, 0, 0]
[0, 2427, 4299, 0, 0, 0, 0]
[0, 2440, 4300, 0, 0, 0, 0]
[0, 2449, 4303, 0, 0, 0, 0]
[0, 2445, 4301, 0, 0, 0, 0]
[0, 2452, 4303, 0, 0, 0, 0]
[0, 2464, 4297, 0, 0, 0, 0]
[0, 2461, 4299, 0, 0, 0, 0]
[0, 2460, 4300, 0, 0, 0, 0]
[0, 2471, 4304, 0, 0, 0, 0]
[0, 2472, 4301, 0, 0, 0, 0]
[0, 2468, 4300, 0, 0, 0, 0]
[0, 2465, 4300, 0, 0, 0, 0]
[0, 2459, 4299, 0, 0, 0, 0]
[0, 2457, 4300, 0, 0, 0, 0]
[0, 2469, 4300, 0, 0, 0, 0]
[0, 2464, 4299, 0, 0, 0, 0]
[0, 2470, 4302, 0, 0, 0, 0]
[0, 2473, 4302, 0, 0, 0, 0]
[0, 2474, 4302, 0, 0, 0, 0]
[0, 2464, 4304, 0, 0, 0, 0]
[0, 2474, 4298, 0, 0, 0, 0]
[0, 2468, 4300, 0, 0, 0, 0

In [22]:
# Stop motion
move_forward(0)

In [200]:
aw(node.unlock())